### 2 inputs 2 outputs - Functional API keras
In this notebook we are going to predict `color` and `marker` from our toy csv file using the functional api.

### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

### Data `prep`.

In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [3]:
if train_df.isnull().values.any():
    train_df.dropna()
if test_df.isnull().values.any():
    test_df.dropna()
    
train_df.head()

,x,y,color,marker
0,-0.765775,1.006452,red,^
1,0.229016,0.094320,red,^
2,0.189987,1.589141,red,^
3,0.580085,0.520488,red,^
4,0.292287,0.317852,red,^


In [4]:
column_transformer = ColumnTransformer([
    ('min_max_scaler', MinMaxScaler(), ['x', 'y']),
    ('one_hot_encoder', OneHotEncoder(), ['color', 'marker']),
]).fit(train_df)

In [5]:
train = column_transformer.transform(train_df)
np.random.shuffle(train) 
test = column_transformer.transform(test_df)
np.random.shuffle(test) 

In [6]:
train[0]

array([0.34843078, 0.01729116, 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        ])

In [7]:
train_input_1 = train[:,0]
train_input_2 = train[:,1]
train_output_1 = train[:,2:-3]
train_output_2 = train[:,-3:]

test_input_1 = test[:,0]
test_input_2 = test[:,1]
test_output_1 = test[:,2:-3]
test_output_2 = test[:,-3:]

> Converting the inputs and outputs to `tensorflow` tensors.

In [8]:
train_input_1_tensors = tf.convert_to_tensor(train_input_1, dtype=tf.float32)
train_input_2_tensors = tf.convert_to_tensor(train_input_2, dtype=tf.float32)
train_output_1_tensors = tf.convert_to_tensor(train_output_1, dtype=tf.int32)
train_output_2_tensors = tf.convert_to_tensor(train_output_2, dtype=tf.int32)

test_input_1_tensors = tf.convert_to_tensor(test_input_1, dtype=tf.float32)
test_input_2_tensors = tf.convert_to_tensor(test_input_2, dtype=tf.float32)
test_output_1_tensors = tf.convert_to_tensor(test_output_1, dtype=tf.int32)
test_output_2_tensors = tf.convert_to_tensor(test_output_2, dtype=tf.int32)

> Creating a `Functional Model` that will take `input1 and input2` to predict `output1 and output2`.

#### Model Architecture.

```python
     (one_hot_vectors)  (one_hot_vectors)
            input1       input2
              |            |
              |            |
        --------------------------
        |      FFN-NETWORK       |
        --------------------------
          |                    |
      (output1)             (output2)
  categorical_entropy     categorical_entropy

```

In [9]:
input_1 = keras.layers.Input(shape=(1,), name="input_1")
input_2 = keras.layers.Input(shape=(1,), name="input_2")

merged_input = keras.layers.Concatenate(axis=1, name="concatenate_layer")([input_1, input_2])
hl_1 = keras.layers.Dense(64, name="hl_1", activation='relu')(merged_input)
hl_2 = keras.layers.Dense(128, name="hl_2", activation='relu')(hl_1)
hl_3 = keras.layers.Dense(64, name="hl_3", activation='relu')(hl_2)
hl_4 = keras.layers.Dense(32, name="hl_4", activation='relu')(hl_3)

output_1 = keras.layers.Dense(6, activation='softmax', name="output_1")(hl_4)
output_2 = keras.layers.Dense(3, activation='softmax', name="output_2")(hl_4)


model = keras.Model(inputs=[input_1, input_2], outputs=[output_1, output_2],
                    name="color_marker_classifier")

model.compile(
    loss = {
        "output_1": keras.losses.CategoricalCrossentropy(from_logits=False),
        "output_2": keras.losses.CategoricalCrossentropy(from_logits=False),
    },
    optimizer = keras.optimizers.Adam(),
    metrics=['acc']
)
model.summary()

Model: "color_marker_classifier"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
concatenate_layer (Concatenate) (None, 2)            0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
hl_1 (Dense)                    (None, 64)           192         concatenate

In [10]:
EPOCHS = 10
VALID_SPLIT = .4
BATCH_SIZE = 32
VERBOSE = 1
history = model.fit(x=[train_input_1_tensors, train_input_2_tensors],
                    y=[train_output_1_tensors, train_output_2_tensors], 
                    epochs=EPOCHS,
                    verbose=VERBOSE, validation_split= VALID_SPLIT,
                    batch_size=BATCH_SIZE
                   )

Epoch 1/10
113/113 [==============================] - 5s 28ms/step - loss: 2.3880 - output_1_loss: 1.5195 - output_2_loss: 0.8686 - output_1_acc: 0.3205 - output_2_acc: 0.6280 - val_loss: 0.8580 - val_output_1_loss: 0.5361 - val_output_2_loss: 0.3219 - val_output_1_acc: 0.8450 - val_output_2_acc: 0.8183
Epoch 2/10
113/113 [==============================] - 1s 5ms/step - loss: 0.6706 - output_1_loss: 0.4049 - output_2_loss: 0.2657 - output_1_acc: 0.8839 - output_2_acc: 0.9002 - val_loss: 0.3508 - val_output_1_loss: 0.2007 - val_output_2_loss: 0.1501 - val_output_1_acc: 0.9458 - val_output_2_acc: 0.9671
Epoch 3/10
113/113 [==============================] - 1s 5ms/step - loss: 0.3347 - output_1_loss: 0.1925 - output_2_loss: 0.1422 - output_1_acc: 0.9331 - output_2_acc: 0.9623 - val_loss: 0.2797 - val_output_1_loss: 0.1601 - val_output_2_loss: 0.1196 - val_output_1_acc: 0.9538 - val_output_2_acc: 0.9654
Epoch 4/10
113/113 [==============================] - 1s 5ms/step - loss: 0.2539 - outp

> Evaluating the `model`.

In [11]:
model.evaluate([test_input_1_tensors, test_input_2_tensors],[test_output_1_tensors, test_output_2_tensors] )

38/38 [==============================] - 1s 4ms/step - loss: 0.1102 - output_1_loss: 0.0636 - output_2_loss: 0.0465 - output_1_acc: 0.9833 - output_2_acc: 0.9842


[0.1101640984416008,
 0.06364080309867859,
 0.046523287892341614,
 0.9833333492279053,
 0.98416668176651]

> Making `predictions`.

In [12]:
predictions = model.predict([test_input_1_tensors, test_input_2_tensors])

In [13]:
color_prediction, marker_prediction = predictions

In [14]:
color_prediction = np.argmax(color_prediction, axis=1)
marker_prediction = np.argmax(marker_prediction, axis=1)

In [15]:
test_output_1_tensors[:5].numpy(), color_prediction[:5]


(array([[0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0]]),
 array([4, 1, 0, 4, 1], dtype=int64))

> The model is making predicting data very well.